In [1]:
import ast
import nbimporter
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import sys
#sys.path.append("C:/Users/Administrator/Desktop/CreditGuard/ML model/")
#print(sys.path)
import model_functions as nb


ModuleNotFoundError: No module named 'model_functions'

In [15]:
df = nb.process_data()
df['age'] = df.apply(lambda row: row['trans_year']-row['dob'].year-((row['trans_month'], row['trans_day'])<(row['dob'].month, row['dob'].day)), axis=1)
df.to_csv("processed.csv", index=False)

In [10]:
df_fraud = df[df['is_fraud'] == 1]
cc_fraud = df_fraud[df_fraud['fraud_type'] == 2]
b_fraud = df_fraud[df_fraud['fraud_type'] == 1]

In [13]:
def draw_graph(G):
    sns.set(style="whitegrid")
    
    plt.figure(figsize=(20, 7))
    pos = nx.spring_layout(G, k=0.15, iterations=20)  # k controls the distance between nodes
    
    # Draw nodes with gradient color based on degree
    node_colors = sns.color_palette("coolwarm", len(G.nodes))
    node_sizes = [(G.degree(n) * 100)+10 for n in G.nodes()]
    
    nodes = nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, alpha=0.9)
    
    # Draw edges with transparency
    edges = nx.draw_networkx_edges(G, pos, alpha=0.4)
    
    # Add labels
    nx.draw_networkx_labels(G, pos, font_size=8, font_color='black')
    
    # Remove axis
    plt.axis('off')
    
    # Title and display
    plt.title('Network Overview', fontsize=20)
    plt.show()

In [16]:
# Example data preparation for network analysis
def build_network_matrix(df):
    # Define relationships (edges) between nodes
    edges = []
    
    for index, row in df.iterrows():
        # Create edges between customer and merchant
        edges.append((row['customer_name'], row['merchant']))
        
        # Create edges between customer and fraud type
        edges.append((row['customer_name'], f"fraud_type_{row['fraud_type']}"))
        
        # Create edges between merchant and category
        edges.append((row['merchant'], row['category']))
    
    return edges

# Get the edges for both fraud types
credit_fraud_edges = build_network_matrix(cc_fraud)
bank_fraud_edges = build_network_matrix(b_fraud)

# Create network graphs using NetworkX
G_credit_fraud = nx.Graph()
G_credit_fraud.add_edges_from(credit_fraud_edges)

G_bank_fraud = nx.Graph()
G_bank_fraud.add_edges_from(bank_fraud_edges)

# Visualize or analyze the networks (e.g., ego networks around specific customers or merchants)


In [ ]:
# Example: Ego network around a specific customer
ego_customer = nx.ego_graph(G_credit_fraud, "Juan Sherman", radius=1)
